In [1]:
import pickle
import subprocess as proc

types = {'INPUT': 'unsigned char',
         'COEFFICIENTS': 'unsigned short',
         'INDEX': 'unsigned long',
         'RESULTS': 'unsigned long long',
         'BAKC_PROJECTED': 'unsigned long long',
         'NORMALIZED': 'unsigned long long'}

pickle.dump(types, open("types.pkl", "wb"))

def generateCythonCode(verbose = False):
    output=[]

    with open('cython_template.py', 'r') as f:
        i = 1
        for line in f:
            code = line
            if 'def' in code:
                if (code.count(',')+1) != code.count('{'):
                    print('#'*100)
                    print(code)
                    print("Warning: Unwrapped type detected at line %i"%i)
            for typ in types:
                indicator = '{%s}'%typ
                if indicator in code:
                    code = code.replace(indicator,types[typ])
            output.append(code)
            i+=1

    with open('functions.pyx', 'w') as f:
        f.write(''.join(output))
        
    if verbose:
        print(''.join(output))

def compileCythonCode():
    print("#"*100)
    print("Compiling ...")
    print("#"*100)
    p1 = proc.run('python setupFinal.py build_ext --inplace', shell=True, capture_output=True)
    print(p1.stdout.decode())
    if 'Finished generating code' in p1.stdout.decode():
        print("#"*100)
        print("Generating report ...")
        print("#"*100)
        p2 = proc.run('cython -a functions.pyx', shell=True, capture_output=True)
        print("Compilation finished!")
        
generateCythonCode()
compileCythonCode()

####################################################################################################
cpdef sample_parallel({INPUT}[::1] img_flat, {COEFFICIENTS}[::1] coeffs, {INDEX}[::1] idx, {RESULTS}[::1] result_flat, unsigned int nThreads):

####################################################################################################
    cdef {INDEX} x, index

####################################################################################################
    cdef {INDEX} x, index

####################################################################################################
Compiling ...
####################################################################################################
Compiling functions.pyx because it changed.
[1/1] Cythonizing functions.pyx
running build_ext
building 'functions' extension
C:\Program Files (x86)\Microsoft Visual Studio\2019\Enterprise\VC\Tools\MSVC\14.27.29110\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -IC:\Users